In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi 
import os 
api = KaggleApi()
os.environ['KAGGLE_WORKING_DIR'] = '.'
api.authenticate()
api.dataset_download_files('paultimothymooney/chest-xray-pneumonia', path='dataset', unzip=True)

In [1]:
import os
data_dir = 'dataset/chest_xray'
train_dir = os.path.join(data_dir,'train')
test_dir  = os.path.join(data_dir,'test')
val_dir   = os.path.join(data_dir,'val')


In [2]:
train_normal_dir =os.path.join(train_dir,'NORMAL')
train_pneu_dir = os.path.join(train_dir,'PNEUMONIA')
val_normal_dir = os.path.join(val_dir,'NORMAL')
val_pneu_dir = os.path.join(val_dir,'PNEUMONIA')
train_normal_images , train_pneu_images = os.listdir(train_normal_dir),os.listdir(train_pneu_dir)
# len(train_normal_images),len(train_Pneu_images)

In [3]:
from torch.utils.data import DataLoader,Dataset
from torchvision import transforms
import torch 
from PIL import Image

class ChestXrayDataset(Dataset):
    def __init__(self,normal_dir,pneu_dir,transform):
        
        self.transform = transform
        self.normal_dir = normal_dir 
        self.pneu_dir = pneu_dir
        self.normal_images = os.listdir(normal_dir)
        self.pneu_images = os.listdir(pneu_dir)

        self.images  = [(0,normal_image) for normal_image in self.normal_images] + [(1,pneu_image) for pneu_image in self.pneu_images]
    
    def __len__(self):
        return len(self.images)
    def __getitem__(self,idx):
        label,img = self.images[idx] 
        img_path = os.path.join(self.normal_dir if label == 0 else self.pneu_dir, img)

        image = Image.open(img_path)
        image = image.convert('RGB')
        image = self.transform(image)
        return image,label

In [4]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])

train_dataset = ChestXrayDataset(train_normal_dir,train_pneu_dir,transform)
val_dataset = ChestXrayDataset(val_normal_dir,val_pneu_dir,transform)

In [5]:

batch_size = 32
train_dataloader = DataLoader(train_dataset,batch_size=batch_size)
val_dataloader = [(x,y) for x, y in  DataLoader(val_dataset,batch_size=batch_size)]


In [6]:
import random

In [7]:
import torchvision.models as models
import torch.nn as nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False 


model.fc = nn.Sequential(
    nn.Linear(in_features=model.fc.in_features, 1024),
    nn.ReLU(),
    nn.BatchNorm1d(1024),
    nn.Dropout(0.4),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.3),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    nn.Dropout(0.2),
    nn.Linear(128, 1)
)

model = model.to(device) 

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
weight = torch.tensor([len(train_normal_images)/len(train_pneu_images)],dtype=torch.float32).to(device)


criterion  = nn.BCEWithLogitsLoss(pos_weight=weight)
lr = 1e-3
optimizer = torch.optim.Adam(model.fc.parameters(),lr=lr,weight_decay=1e-5)
epochs = 100



In [16]:
for epoch in range(epochs):
    batch_count = 0
    epoch_loss = 0 
    for X,y in train_dataloader:
        model.train()
        batch_count += 1
        optimizer.zero_grad()
        X = X.to(device)
        y = y.float().to(device)
        y = y.unsqueeze(1)
        y_pred = model(X).to(device)
        loss = criterion(y_pred,y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        model.eval()
        with torch.no_grad():
            valX,valy = random.sample(val_dataloader,1)[0]
            valX = valX.to(device)
            valy = valy.float().to(device)
            valy = valy.unsqueeze(1)
            val_preds = model(valX)
            val_loss = criterion(val_preds,valy)

        if batch_count % 100 != 0:
            print(f'Epoch : {epoch}/{epochs} , Batch: {batch_count} , Loss : {loss.item()} , Val loss: {val_loss}')

    print(f'Epoch : {epoch} , Average loss: {epoch_loss/batch_count}')

Epoch : 0/100 , Batch: 1 , Loss : 0.6453663110733032 , Val loss: 0.4507593512535095
Epoch : 0/100 , Batch: 2 , Loss : 0.665534496307373 , Val loss: 0.4880267381668091
Epoch : 0/100 , Batch: 3 , Loss : 0.6878052353858948 , Val loss: 0.5126184225082397
Epoch : 0/100 , Batch: 4 , Loss : 0.6490633487701416 , Val loss: 0.5098622441291809
Epoch : 0/100 , Batch: 5 , Loss : 0.6685536503791809 , Val loss: 0.47380125522613525
Epoch : 0/100 , Batch: 6 , Loss : 0.6851694583892822 , Val loss: 0.4585728645324707
Epoch : 0/100 , Batch: 7 , Loss : 0.7086092233657837 , Val loss: 0.46598881483078003
Epoch : 0/100 , Batch: 8 , Loss : 0.6378028988838196 , Val loss: 0.4766787886619568
Epoch : 0/100 , Batch: 9 , Loss : 0.6390016078948975 , Val loss: 0.4996042251586914
Epoch : 0/100 , Batch: 10 , Loss : 0.6701545715332031 , Val loss: 0.518699049949646
Epoch : 0/100 , Batch: 11 , Loss : 0.6691068410873413 , Val loss: 0.5051045417785645
Epoch : 0/100 , Batch: 12 , Loss : 0.6425657272338867 , Val loss: 0.504904

KeyboardInterrupt: 